<a href="https://colab.research.google.com/github/CBravoR/AdvancedAnalyticsLabs/blob/master/notebooks/python/Lab_7_Keras_and_Shallow_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Keras and Shallow Networks

In this lab we will create our first neural network. It will be a shallow one, over a small dataset, so it is not a good example of a real-life problem where neural networks shine, but it will be a great way to understand the network architectures and what can we do with the model.

In this lab, and in general in future examples, we will closely follow the book "Deep Learning with Python" by Chollet. The library has copies of the book.

## Loading the data

We will import one of the data examples from Keras itself: Prices in Boston.

But first, what is Keras?

[Keras](https://keras.io/) is a part of the Deep Learning stack, providing an easier way to create models. Deep Learning requires a set of very sophisticated optimizers and computational calculations, so it can be tricky to program directly. Luckily for us, we can simply use Keras to call the more sophisticated programs that will do the heavy lifting for us.

Technically, Keras is a "front-end", while the calculation engine will be a "backend". Those backends will actually call a backend of their own which will communicate with the machine itself.

First, we need to configure our backend. Google has created the excellent "tensorflow" backend, which is able to handle many sophisticated neural networks, and it can leverage the use of a GPU (a video card) for great speedups. It has also made available GPUs for us to play with, so

If you want to configure your own environment in your PC, and have an Nvidia GPU at home or your laptop, please follow the following steps:

1. Install Anaconda Python from [here](https://www.anaconda.com/distribution/).

2. Follow the steps [here](https://towardsdatascience.com/tensorflow-gpu-installation-made-easy-use-conda-instead-of-pip-52e5249374bc) to configure your Python environment with tensorflow.

We will continue using Google Colab's environment, so you don't need to do this now. What we need to do is to change our execution environment to a GPU one. For that:

- Click Runtime
- Select "Change runtime type"
- In the menu, in the combo box below "Hardware Accelerator" select GPU.

There is an even more powerful environment: TPU, but this requires a change in coding so we will omit it until the very last lab. Now we are ready to use Keras!

## Keras Sequential API

The easiest model that can be constructed in Keras is a sequential model. This is the typical neural network with one input layer, and one output layer, and a linear sequence of as many layers as we want in-between. The alternative is the Keras API, but for most applications a sequential model is enough. A tutorial for the sequencial API can be found [here](https://keras.io/getting-started/sequential-model-guide/).

First let's load the data.

In [0]:
import tensorflow.keras as keras
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [0]:
print('Size of the training set: ', train_data.shape)
print('Size of the test set: ', test_data.shape)

This is a very small dataset, with only 404 cases for training, 102 for testing, 13 variables, and 1 target representing the median house price in Boston for the neighbourhood. The variables can be found [here](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html).

We have one problem: All variables are in different scales. The following code normalizes the data.

In [0]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

Now we can create our model. We want to build the following architecture:

![](https://drive.google.com/uc?export=view&id=1CfIe77tmoQrh4QEBFZtabYpiM3rdNOOs)

This example uses the optimizer RMSProp, which is great for sequences. There are many optimizers, but as a rule of thumb, use test SGD, Adam, and RMSProp. **All of them solve the same backpropagation problem**, they are just tuned to certain data structures. A great comparison of different optimization algorithms can be found [here](http://ruder.io/optimizing-gradient-descent/index.html).

To construct the model, we create a "model" object using Keras API. This is simply a container which will hold the general structure of our model, and we will add layers to it. Let's import the layers and the models helper functions.

In [0]:
from keras import models
from keras import layers

Now, let's build an empty sequential model. Let's name it "model" (yeah, original).

In [0]:
model = models.Sequential()

This is just an empty model, it does not have any layers! Let's add the first hidden layer. For each model object that we have, the function "add" will append a layer stacking one on top of the other.

Remember, we want to add a fully-connected (Dense)layer, with 64 hidden neurons and ReLU activation function. We also need to tell this first layer what input to expect, we do this with the "input_shape" option. In this case, we are giving it a vector of size 13.

In [0]:
model.add(layers.Dense(64, # The number of neurons
                      activation = 'relu', # The type of activation
                      input_shape = (13,)  # The input shape. It is only necessary for the layer connected to the input.
                      ))

You can ignore the warning: it will be fixed in the next version of Keras.

Keras supports many activations. The list can be seen [here](https://keras.io/activations/). It also has many types of layers! The list [here](https://keras.io/layers/about-keras-layers/). Don't worry about this now, we will use many in the coming labs.

Let's add one more Dense layer.

In [0]:
model.add(layers.Dense(64, activation = 'relu'))

Note that now we didn't need to specify inputs and outputs, Keras will deduce them from the previous layer (thus the sequential structure!).

Now, let's add the final output layer. This is simply a Dense layer, of size 1 and linear activation (**not ReLU!**). As this is a regression problem, we want outputs between $(-\infty,\infty)$.

In [0]:
model.add(layers.Dense(1, activation = 'linear'))

With the model ready, the last step is to tell it how to train it. For this, we tell it to "compile it", and give it an optimizer. Note the error is the Minimum Square Error (MSE), which is used for regressions. We will see several more for different applications, but the list can be checked [here](https://keras.io/losses/).

In [0]:
model.compile(optimizer = 'rmsprop', loss = 'mse')

The optimizer has a learning rate parameter that needs to be set. For this lab we will ignore it and just use the default, but we will dig much deeper into this in the future.

We can take a look at the model structure by running the following command:

In [0]:
model.summary()

Or we can show it much more nicely in a picture.

In [0]:
import matplotlib.pyplot as plt
from keras.utils import plot_model
from IPython.display import Image
%matplotlib inline

plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
Image(retina=True, filename='model.png')

## Training and testing

Now we can simply train the model! It follows the same fitting structure of scikit-learn, with a 'fit' function we will use to tell the model the data it needs to train on.

In [0]:
history = model.fit(train_data, train_targets, epochs = 100, batch_size=128, 
                    validation_split= 0.25)

Done! Let's examine the command in detail:

- The first two arguments give us the traning data and the training targets. These should always be Numpy arrays, or other compatible tensors.

- epochs: Neural networks are trained using batches of data, which are shuffled and then fed back to the network. Each full pass of the data is called an "Epoch". We will use the number of epochs to control overfitting with the validation sample.

- batch_size: For big data models, it is not possible to train the model with all data at once, as we could be done here. The reason for that is that the model may not fit in memory, or that we need to stream data from an online source. The traditional way is to train in "batches". Recommended batches size are between 32 (for big applications) to 256 (for small problems). In general, we want to keep it small to create "diversity" in the data.

- validation_split: As this model is *very* prone to overfitting, we need to control it somehow. The validation data will help us with this. We will stop training as soon as we start worsening the validation sample. That will be the cut for the epochs in later iterations. We use a 25% split for validation cases. Another alternative is to provide a new set of data to use as validation data, we would use  "validation_data = (val_train, val_target)" instead of this split commmand.


The output we currently have gives us a training loss and a validation loss. This is measured in MSE. But, should we keep training? Are we making any gains?

The following code allows us to visualize the regression metrics.

In [0]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Is it worth it to retrain? If you run the "fit" command again, Keras will optimize *over the parameters already found*, so we would actually be improving the result over the training set (but potentially worsening it over the validation set!). The goal is to find a number of epochs that gives us the best result. 

In our case: the validation loss and the training loss are still going down! We can continue training. Let's give it 100 more epochs.

In [0]:
history = model.fit(train_data, train_targets, epochs = 100, batch_size=128, validation_split= 0.25)

In [0]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Still going down! As a self-study exercise, keep training the model until you see an increase in validation loss. You should train until this happen, so keep an eye in your models!

And how did we do in the test set? The "evaluate" internal function allows us to validate on new data.

In [0]:
model.evaluate(test_data, test_targets)

We can use this number to compare against other models. Or take the square root to get an idea of the average error to be incurred, and then decide.

Alternatively, if we want to get the value of the predictions, we can use the following command, which returns a prediction over the test data.

In [0]:
model.predict(test_data)

And that's it! We have our first shallow neural network trained. In future labs we will be adding complexity to this by for example using more complex text-based embeddings, images, etc.

# Self Study

Go through this example again, but now following Chapter 3.6 of Chollet's book. There you'll learn how to use Cross-Validation to model this example. Modify the activations and the size of the network. Try, for example:

- Using SeLU instead of ReLU
- Adding one more hidden layer
- Changing the number of neurons from 64 to 128
- ...etc

Can you reach a better predictive capability over the test set? Creating an architecture is an art, so be creative and do your best effort!